In [ ]:
#default_exp core.vase_model

In [ ]:
#export
import torch
from torch import nn
from cmaes import CMA
import numpy as np

from vase.core.models import FCEncoder, FCDecoder, Encoder, Decoder, EnvironmentInference
from vase.core.utils import rec_likelihood, disable_gradient, kl_div_stdnorm

# Vase Model

In [ ]:
class VASE(nn.Module):
    def __init__(self, 
        encoder_type: type,
        decoder_type: type,
        latents: int,
        final_size: int,
        max_envs: int,
        env_optim: type, 
        env_lr: float,
        replay_batch_size: int,
        tau: int,
        kappa: float,  
        lam: float,
        lam_1: float,
        lam_2: float, 
        Tau: float,
        used_epochs: int,
        used_delta: float,
        used_sigma: float,
        device: str
    ):
        super().__init__()
        self.latents = latents
        self.max_envs = max_envs
        self.final_size = final_size
        self.device = device
        self.encoder = encoder_type(self.latents)
        self.decoder = decoder_type(self.latents, self.max_envs)
        self.old_encoder = encoder_type(self.latents)
        self.old_decoder = decoder_type(self.latents, self.max_envs)
        self.copy_and_freeze()
        self.encoder.to(self.device), self.decoder.to(self.device), self.old_encoder.to(self.device), self.old_decoder.to(self.device)
        self.steps = 0
        self.tau = tau
        self.replay_batch_size = replay_batch_size
        self.env_net = EnvironmentInference(self.max_envs, self.final_size)
        self.m = 0
        self.env_optim = env_optim(params=self.env_net.parameters(), lr=env_lr)
        self.env_loss = nn.CrossEntropyLoss()
        self.rec_losses = []
        self.kappa = kappa
        self.masks = []
        self.lam = lam
        self.lam_1 = lam_1
        self.lam_2 = lam_2
        self.Tau = Tau
        self.used_epochs = used_epochs
        self.used_delta = used_delta
        self.used_sigma = used_sigma

    def forward(self, x):
        batch_size = x.shape[0]
        self.steps += batch_size

        mu, logvar, final = self.encoder(x)
        s_hat, env_logits = self.get_likely_env(final)
        if self.training:
            z = self.reparam(mu, logvar)
        
        a, alphas = self.get_mask(z)
        masked_z, masked_mu, masked_logvar = self.apply_mask(z, a, mu, logvar)

        s = self.infer_env(x, batch_size, masked_z, s_hat, a)
        
        rec_x = self.decoder(masked_z, self.int_to_vec(s, batch_size))

        x_halu, s_halu = self.sample_old(s)
        with torch.no_grad():
            mu_halu_old, logvar_halu_old, final_halu_old = self.old_encoder(x_halu)
            z_halu_old = self.reparam(mu_halu_old, logvar_halu_old)
            a_halu_old = torch.tensor([self.masks[s] for s in list(s_halu)])
            masked_z_halu_old, masked_mu_halu_old, masked_logvar_halu_old = self.apply_mask(z_halu_old, a_halu_old)
        
        mu_halu, logvar_halu, final_halu = self.encoder(x_halu)
        z_halu = self.reparam(mu_halu, logvar_halu)
        a_halu, alphas_halu = self.get_mask(z_halu)
        masked_z_halu, masked_mu_halu, masked_logvar_halu = self.apply_mask(z_halu, a_halu, mu_halu, logvar_halu)
        rec_x_halo = self.decoder(masked_z_halu, s_halu)

        self.train_env_network(env_logits, s, final_halu_old, s_halu)

        if self.tau and self.steps > self.tau:
            self.copy_and_freeze()

        return rec_x, masked_mu, masked_logvar, x_halu, masked_z_halu_old, rec_x_halo, masked_z_halu

    def get_mask(self, z):
        std, mean = torch.std_mean(z, dim=0)
        std = std[:,None]
        mean = mean[:, None]
        logvar = torch.log(std.pow(2))
        alphas = kl_div_stdnorm(mean, logvar)
        alphas[alphas < self.lam_1] = 0
        alphas[alphas > self.lam_2] = 1
        a = alphas < self.lam
        return a, alphas
    
    def get_likely_env(self, final):
        env_logits = self.env_network(final)
        avg_env_logits = torch.mean(env_logits, dim=0)
        return torch.argmax(avg_env_logits), env_logits
    
    def reparam(self, mu, logvar):
        eps = torch.randn(logvar.shape).to(self.devicedevice)
        std = (0.5 * logvar).exp()
        return mu + std * eps
    
    def infer_env(self, x, batch_size, masked_z, s_hat, a):
        with torch.no_grad():
            rec_x = self.decoder(masked_z, self.int_to_vec(s_hat, batch_size))
        rec_loss = torch.mean(rec_likelihood(x, rec_x))
        
        if self.m == 0:
            self.init_env(rec_loss, a)
            return 0
        
        if self.m == self.max_envs - 1:
            self.update_env(rec_loss, a, self.m)
            return self.m
        
        if rec_loss > self.kappa * self.rec_losses[s_hat]:
            self.m+=1
            self.init_env(rec_x, a)
            return self.m
        
        if a != self.masks[s_hat]:
            u = self.get_used_mask(s_hat) 
            if a * u != self.masks[s_hat] * u:
              self.m+=1
              self.init_env(rec_x, a)
              return self.m

        self.update_env(rec_loss, a, s_hat)
        return s_hat 
        
    def init_env(self, rec_x, a):
        if self.tau is None:
            self.copy_and_freeze()
        self.rec_losses.append(rec_x)
        self.masks.append(a)
    
    def update_env(self, rec_loss, a, s):
        self.rec_losses[s] = rec_loss
        self.masks[s] = a
    
    def get_used_mask(self, x, batch_size, z, s_hat):
        optimizer = CMA(mean=np.zeros(self.latents), sigma=self.used_sigma)
        s_vec = self.int_to_vec(s_hat, batch_size)
        for generation in range(self.used_epochs):
            solutions = []
            for _ in range(optimizer.population_size):
                sigma_a = optimizer.ask()
                with torch.no_grad():
                    sigma_t = torch.tensor(x, dtype=torch.float)
                    sigma_t = torch.abs(sigma_t)
                    eps = torch.randn(sigma_t.shape[0]) * sigma_t
                    z_e = (1-self.used_delta) * z + (self.used_delta + eps)
                    rec_x = self.decoder(z_e, s_vec)
                    rec_loss = torch.mean(rec_likelihood(x, rec_x))
                    sum_sigma = torch.sum(sigma_t)
                    loss = rec_loss - sum_sigma
                value = loss.detach().numpy()
                solutions.append((sigma_a, value))
            optimizer.tell(solutions)
        sigma_p_a = sorted(solutions, key=lambda pair: pair[1], reverse=True)[0][0]
        sigma_p_t = torch.tensor(sigma_p_a, dtype=torch.float).abs()
        return sigma_p_t < self.Tau

    
    def int_to_vec(self, i, size):
        return torch.ones([size], dtype=torch.int64) * i
    
    def apply_mask(z, a, mu, logvar):
        #TODO: verify it broadcasts correctly in both cases
        std_norm = torch.randn(z.shape)
        masked_z = a * z + (~a) * std_norm # "reparam" trick again
        masked_mu = mu * a
        masked_logvar = logvar * a
        return masked_z, masked_mu, masked_logvar
    
    def copy_model(self, old_model, cur_model):
        old_model.load_state_dict(cur_model.state_dict())
    
    def freeze_model(self, model):
        disable_gradient(model)
    
    def copy_and_freeze(self):
        self.copy_model(self.old_encoder, self.encoder)
        self.freeze_model(self.old_encoder)
        self.copy_model(self.old_decoder, self.decoder)
        self.freeze_model(self.old_decoder)
    
    def train_env_network(self, env_logits, s, final_halu_old, s_halu):
        self.env_optim.zero_grad()
        cur_loss = self.env_loss(env_logits, s) #don't know if dims work here
        env_logits_halu = self.env_net(final_halu_old)
        replay_loss = self.env_loss(env_logits_halu, s_halu)
        loss = cur_loss + replay_loss
        loss.backward()
        self.env_optim.step()

In [ ]:
latents=8
final_size=50
max_envs=7
env_optim=torch.optim.Adam
env_lr=6e-4
replay_batch_size=64
tau=None
kappa=1.5
lam=1.5,
lam_1=.01, 
lam_2=3, 
Tau=None, 
used_epochs=50 
used_delta=.95
used_sigma = 1.3
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
vase = VASE(FCEncoder, FCDecoder, latents, final_size, max_envs, env_optim, env_lr, replay_batch_size, tau, kappa, lam, lam_1, lam_2, Tau, used_epochs, used_delta, used_sigma, device)